In [1]:
import pandas as pd


In [58]:
df_distances = pd.read_csv("../../results_demo/T50/FFT/Group_2/Minkowski/distance_estimator.csv")
df_distances

,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,sequence_53082,55.0,sequence_19402,43.0,9488.733257
1,sequence_53082,55.0,sequence_20054,49.0,9907.365945
2,sequence_53082,55.0,sequence_20048,39.8,9956.539490
3,sequence_53082,55.0,sequence_20036,52.9,9425.898334
4,sequence_53082,55.0,sequence_19525,48.8,9219.552734
...,...,...,...,...,...
67855,sequence_19401,47.4,sequence_53056,52.0,3783.409735
67856,sequence_19401,47.4,sequence_19347,58.9,3901.046247
67857,sequence_19401,47.4,sequence_19379,50.3,2555.097611
67858,sequence_19401,47.4,sequence_19348,46.7,6147.325333


In [59]:
#filtramos las distancias para trabajar con solo una parte de ellas -> nos interesan solo las más cercanas
import numpy as np

q1 = np.quantile(df_distances['distance'], .25)
q3 = np.quantile(df_distances['distance'], .75)

print(q3, q1)
IQR = q3 - q1
IQR = 1 * IQR

low_freq = q1 - IQR
high_freq = q3 + IQR

print(low_freq, high_freq)

df_filter = df_distances.loc[df_distances['distance'] <= q3]
df_filter = df_filter.reset_index()

8078.620991076636 5970.00106976452
3861.381148452405 10187.240912388752


In [60]:
df_filter

,index,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,16,sequence_53082,55.0,sequence_53062,49.1,6687.702985
1,27,sequence_53082,55.0,sequence_53085,51.2,7661.106185
2,32,sequence_53082,55.0,sequence_19369,47.9,7520.527732
3,38,sequence_53082,55.0,sequence_20064,47.3,6793.094573
4,41,sequence_53082,55.0,sequence_19504,50.6,7689.858708
...,...,...,...,...,...,...
50891,67855,sequence_19401,47.4,sequence_53056,52.0,3783.409735
50892,67856,sequence_19401,47.4,sequence_19347,58.9,3901.046247
50893,67857,sequence_19401,47.4,sequence_19379,50.3,2555.097611
50894,67858,sequence_19401,47.4,sequence_19348,46.7,6147.325333


In [61]:
import community as community_louvain
import networkx as nx

In [62]:
graph_data = nx.Graph()

id_list1 = df_filter['id_seq1'].to_list()
id_list2 = df_filter['id_seq2'].to_list()

nodos = id_list1 + id_list2
nodos = list(set(nodos))

#cargar nodos
for node in nodos:
    graph_data.add_node(node)

#cargar aristas
for i in range(len(df_filter)):
    graph_data.add_edge(df_filter['id_seq1'][i],
        df_filter['id_seq2'][i],
        weigth=df_filter['distance'][i]
    )

In [63]:
partition = community_louvain.best_partition(graph_data)
modularity_value = round(community_louvain.modularity(partition, graph_data), 3)

print("Modularity: ", modularity_value)

Modularity:  0.069


In [64]:
#obtenemos los grupos
matrix_group = []
for element in partition:
    row = [element, partition[element]]
    matrix_group.append(row)
results = pd.DataFrame(matrix_group, columns=['id', 'label'])

results.to_csv("../../results_demo/T50/FFT/Group_2/Minkowski/graph_clustering_sequences.csv", index=False)